In [1]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import decode_predictions
from keras.applications.resnet50 import preprocess_input
import numpy as np
from os import walk
import json
from image_generator import ImageGenerator

Using TensorFlow backend.


In [2]:
def get_train_test_labels():
    train_labels = dict()
    test_labels = dict()

    with open('market_attribute.json') as json_file:
        data = json.load(json_file)
        train = data['train']
        test = data['test']

        for i, image_index in enumerate(train['image_index']):
            img_labels = []
            for key in train.keys():
                if key != 'image_index':
                    img_labels.append(train[key][i])
            train_labels[image_index] = np.array(img_labels)
            
        for i, image_index in enumerate(test['image_index']):
            img_labels = []
            for key in train.keys():
                if key != 'image_index':
                    img_labels.append(train[key][i])
            test_labels[image_index] = np.array(img_labels)
            
    return train_labels, test_labels

def get_train_test_images():
    files = []
    for (dirpath, dirnames, filenames) in walk("./Market-1501"):
        files.extend(filenames)
        break
        
    print('number of images:', len(files))
    
    files.sort()
    
        
    train_images = []
    test_images = []
    with open('market_attribute.json') as json_file:
        data = json.load(json_file)
        train_indexes = data['train']['image_index']
        test_indexes = data['test']['image_index']
        for i, file in enumerate(files):
            if file[:4] in train_indexes:
                train_images.append(file)
            elif file[:4] in test_indexes:
                test_images.append(file)
                
    return (train_images, test_images)

from keras.utils import Sequence
from keras_preprocessing import image
from keras.applications.resnet50 import preprocess_input

class ImageGenerator(Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(224, 224), n_channels=3,
                 n_classes=10, shuffle=True):

        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, * self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        for i, image_path in enumerate(list_IDs_temp):
            img = image.load_img("Market-1501/" + image_path, target_size=(224, 224, 3))
            
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)

            img_index = image_path[:4]

            labels = np.delete(self.labels[img_index], 0) - 1
            
            X[i, ] = img_array
            y[i, ] = labels
            
        return X, y

In [3]:
attributes_length = 26
epochs = 60
batch_size = 32

train_images, test_images = get_train_test_images()
train_labels, test_labels = get_train_test_labels()

ig_train = ImageGenerator(train_images, train_labels, n_classes=26)
ig_test = ImageGenerator(test_images, test_labels, n_classes=26)

number of images: 25260


In [4]:
model = Sequential()
model.add(ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(attributes_length, activation='sigmoid'))

W1208 13:33:12.629361 139776534595392 deprecation.py:506] From /home/david/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:211: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/home/david/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W1208 13:33:28.427335 139776534595392 deprecation.py:506] From /home/david/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `kee

In [5]:
for layer in model.layers[0].layers:
    layer.trainable = False

In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               51380736  
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 26)                13338     
Total params: 74,983,834
Trainable params: 51,395,098
Non-trainable params: 23,588,736
_______________________________________________________

In [8]:
for a in ig_train:
    X,y = a
    print(X.shape, y.shape, y[0])
    break

(32, 224, 224, 3) (32, 26) [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0]


In [9]:
history = model.fit_generator(generator=ig_train, 
                              validation_data=ig_test, 
                              epochs=epochs)

Epoch 1/60
 11/392 [..............................] - ETA: 32:53 - loss: 0.9125 - acc: 0.5531

KeyboardInterrupt: 